# Introduction to Options
#### Lucien Chen

## Definitions
- *Derivative* - a financial instrument whose value depends on one (or more) underlying variables
- *Strike* - the price fixed by the options contract
- *Maturity* - also known as the expiration date, the day the contract expires
- *Premium* - price you pay per share for an option broken down into intrinsic value and extrinsic, or time, value
- *Spot* - the price of an asset that can be sold or bought immediately, the current trading price
- *Intrinsic value* - the positive difference between the spot and the strike (call); the positive difference between the strike and the spot (put)
- *Time value* - value of an option based on it's DTE (days to expiration) due to the possibility of an increase in instrinsic value

## What are Options?
 - Options are financial derivatives (FDs), i.e futures, forward contracts, swaps, etc., that give a buyer the right, but not an obligation, to buy a sell a security at a certain price, known as the strike, at a certain date
 - Call options are an agreement to buy shares of stock (usually in quantities of 100) at a certain price by a certain date
 - Put options are an agreement to sell shares of a stock (usually in quantities of 100) at a certain price by a certain date
 - American vs. European options: European options can only be exercised on the expiration date where as American options can be exercised anytime before
 - Options and other FDs are commonly used for hedging, speculation, arbitrage and C-suite compensation
 - Hedgers aim to reduce risk, speculators aim to profit from betting on future movements in the underlying, and arbitrageurs aim to profit from the bid-ask spread on options

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import yfinance as yf

In [3]:
cols = ['European Call', 'European Put', 'American Call', 'American Put']
spot = ['+', '-', '+', '-']
strike = ['-', '+', '-', '+']
maturity = ['?', '?', '+', '+']
vega = ['+', '+', '+', '+']
rho = ['+', '-', '+', '-']
dividends = ['-', '+', '-', '+']
option_properties = pd.DataFrame(columns = cols)
option_properties.loc['Current Price'] = spot
option_properties.loc['Strike'] = strike
option_properties.loc['Time to Expiration'] = maturity
option_properties.loc['Volatility'] = vega
option_properties.loc['Risk-free Rate'] = rho
option_properties.loc['Future Dividends'] = dividends
option_properties

,European Call,European Put,American Call,American Put
Current Price,+,-,+,-
Strike,-,+,-,+
Time to Expiration,?,?,+,+
Volatility,+,+,+,+
Risk-free Rate,+,-,+,-
Future Dividends,-,+,-,+


###### **Assuming variables increase*

__*+*__ represents an increase in value

__*-*__ represents a decrease in value

__*?*__ represents an ambiguous change

In [4]:
rf = yf.Ticker('^IRX').history().Close.iloc[-1] / 100 

The Black-Scholes option formula is a formula that is commonly used to price *European* options.

Assumptions:
 - Dividends are not paid out over the course of the options life
 - Markets are random
 - Zero transaction costs
 - Risk-free rate and volatility are constant
 - Returns are normally distributed
 - Option can only be exercised at expiration
 
$$C=N(d_1)S_t - N(d_2)Ke^{-rt}$$
$$P=N(d_1)K - N(d_2)S_te^{-rt}$$
$$d_1=\frac{\ln{\frac{S_t}{K}} + (r + \frac{\sigma^2}{2})t}{\sigma\sqrt{t}}$$
$$d_2=d_1 - \sigma\sqrt{t}$$

C = call option price

P = put option price

N = cumulative distribution function (cdf) of the normal distribution

$S_t$ = spot price of the underlying

K = strike price

r = risk-free rate

t = time to expiration

$\sigma$ = volatility of the underlying

In [5]:
def bs_call(S, K, r, t, sigma):
    N = norm.cdf
    d1 = (np.log(S/K) + ((r + (sigma**2 / 2)) * t)) / (sigma * np.sqrt(t))
    d2 = d1 - (sigma * np.sqrt(t))
    return N(d1) * S - N(d2) * K * (np.e ** (-r * t))

def bs_put(S, K, r, t, sigma):
    N = norm.cdf
    d1 = (np.log(S/K) + ((r + (sigma**2 / 2)) * t)) / (sigma * np.sqrt(t))
    d2 = d1 - (sigma * np.sqrt(t))
    return N(d1) * K - N(d2) * S * (np.e ** (-r * t))

In [11]:
def create_graph(K, r, t, sigma):
    S = np.arange(50, 151, 0.1)
    calls = [bs_call(s, K, r, t, sigma) for s in S]
    puts = [bs_put(s, K, r, t, sigma) for s in S]
    fig = plt.figure(figsize=(8,4))
    plt.plot(S, calls, label="Call Option Value")
    plt.plot(S, puts, label="Put Option Value")
    plt.xlabel("$S_0$")
    plt.ylabel("Value ($)")
    plt.title("Black-Scholes Pricing Demo")
    plt.legend()

In [13]:
import ipywidgets as widgets
from IPython.display import display

In [12]:
w = widgets.interact(create_graph, K=widgets.IntSlider(min=50, max=150), r=widgets.fixed(rf), t=widgets.IntSlider(min=1, max=365), sigma=widgets.FloatSlider(min = 0.05, max=1, step=0.05))
display(w)

interactive(children=(IntSlider(value=50, description='K', max=150, min=50), IntSlider(value=1, description='t…

<function __main__.create_graph(K, r, t, sigma)>

Sources: 
- investopedia.com
- *Options, Futures, and Other Derivatives (9th Edition), Hull*